In [1]:
#import graphlab
import pandas as pd
import numpy as np
import glob
import math
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
fileList= glob.glob("../data/*.*")
fileList

['../data\\RegularSeasonCompactResults.csv',
 '../data\\RegularSeasonDetailedResults.csv',
 '../data\\sample_submission.csv',
 '../data\\Seasons.csv',
 '../data\\Teams.csv',
 '../data\\TourneyCompactResults.csv',
 '../data\\TourneyDetailedResults.csv',
 '../data\\TourneySeeds.csv',
 '../data\\TourneySlots.csv']

In [3]:
def calcExp(rating_1,rating_2):
    return (1.0 / (1.0 + pow(10, ((rating_2 - rating_1) / 400))))

def modifyRating(rating, expected, actual, kfactor):
    calc = (kfactor * (actual - expected))
    return calc

In [4]:
def getTeamFrame(i):
    qq= seasonTeams_orig.set_index(np.repeat(i,len(seasonTeams_orig)), append=True)
    qq['rating']= np.NAN
    qq.index.names= ['Team_Id','round']
    qq.head(4)
    return qq

# Read in Tournament Seeds

In [5]:
#column_type_hints=[long,long,long,long,long,long,str,long]
#tSeed= graphlab.SFrame.read_csv(fileList[7], verbose=False)
tSeed= pd.read_csv(fileList[7], verbose=False)

# Read in the `Compact Tournament` Game Data

In [6]:
column_type_hints=[long,long,long,long,long,long,str,long]
#tscr= graphlab.SFrame.read_csv(fileList[5], verbose=False)
tscr= pd.read_csv(fileList[5], verbose=False)

# Read in the Compact Game Data

In [7]:
column_type_hints=[long,long,long,long,long,long,str,long]
#rscr= graphlab.SFrame.read_csv(fileList[0], verbose=False)
rscr= pd.read_csv(fileList[0], verbose=False)

In [8]:
column_type_hints=[long,str]
#teams= graphlab.SFrame.read_csv(fileList[4], verbose=False)
teams= pd.read_csv(fileList[4], verbose=False)

In [9]:
teams.head(2)

,Team_Id,Team_Name
0,1101,Abilene Chr
1,1102,Air Force


# Select a season to analyze

In [10]:
selectedSeason=1985

### Filter the `GAMES` data to the selected Season

In [11]:
seasonData= rscr[rscr['Season']==selectedSeason]
seasonData.sort_values(by=['Daynum','Wteam'])
seasonData.reset_index(inplace=True)
seasonData= seasonData.rename(columns={'index':'gameID'})
seasonData.head(4)

,gameID,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,0,1985,20,1228,81,1328,64,N,0
1,1,1985,25,1106,77,1354,70,H,0
2,2,1985,25,1112,63,1223,56,H,0
3,3,1985,25,1165,70,1432,54,H,0


### Filter the `TEAMS` data to the selected Season

In [12]:
w= seasonData['Wteam'].unique()
l= seasonData['Lteam'].unique()
seasonTeams= np.append(w,l)
seasonTeams= pd.Series(list(set(seasonTeams)), name='team')
seasonTeams= seasonTeams.to_frame()
seasonTeams= teams.merge(seasonTeams, left_on='Team_Id', right_on='team')[teams.columns]
seasonTeams.set_index('Team_Id', drop=False, inplace=True)
seasonTeams_orig= seasonTeams.copy()
seasonTeams.head(2)

,Team_Id,Team_Name
Team_Id,,
1102,1102,Air Force
1103,1103,Akron


### Create a `TALL` version of the games data

In [13]:
w= seasonData[['gameID','Season','Daynum','Wteam']].rename(columns={'Wteam':'team'})
w['outcome']= 1
l= seasonData[['gameID','Season','Daynum','Lteam']].rename(columns={'Lteam':'team'})
l['outcome']= 0

seasonDataTeam= w.append(l).sort_values(by=['gameID','team']).reset_index(drop=True).reset_index().rename(columns={'index':'teamGameID'})
seasonDataTeam= seasonDataTeam.set_index(['gameID','outcome'],drop=False)
seasonDataTeam_orig= seasonDataTeam.copy()
seasonDataTeam.head(4)

teamGameID  gameID  Season  Daynum  team  outcome
gameID outcome                                                   
0      1                 0       0    1985      20  1228        1
       0                 1       0    1985      20  1328        0
1      1                 2       1    1985      25  1106        1
       0                 3       1    1985      25  1354        0

In [14]:
seasonTeams['rating']= 0.
seasonData.sort_values(by=['Daynum','Wteam'], inplace=True)
for i in range(12):
    seasonTeams['games']= 0
    seasonTeams['wins']= 0
    
    #store last round
    seasonTeams['rating_'+str(i)]= seasonTeams['rating']

    #Shrinkage
    seasonTeams['rating']= (seasonTeams['rating']-0)/2+0
    


    for loc in range(len(seasonData)):

        x= seasonData.iloc[loc]

        rA= seasonTeams.ix[x['Wteam']]['rating']
        rB= seasonTeams.ix[x['Lteam']]['rating']
        eA= calcExp(rA,rB)
        eB= 1-eA
        rA_n= modifyRating(rA, eA, 1, max(50-10*i,5))
        rB_n= modifyRating(rB, eB, 0, max(50-10*i,5))
        if rA_n != -rB_n:
            print(rA,rB,eA,eB,rA_n,rB_n)

        seasonTeams.loc[x['Wteam'],('rating')]+= rA_n
        seasonTeams.loc[x['Wteam'],('games')]+= 1
        seasonTeams.loc[x['Wteam'],('wins')]+= 1
        seasonDataTeam.loc[(x['gameID'],1),('rating_pre')]= rA
        seasonDataTeam.loc[(x['gameID'],1),('rating_post')]= rA+rA_n
        
        seasonTeams.loc[x['Lteam'],('rating')]+= rB_n
        seasonTeams.loc[x['Lteam'],('games')]+= 1
        seasonDataTeam.loc[(x['gameID'],0),('rating_pre')]= rB
        seasonDataTeam.loc[(x['gameID'],0),('rating_post')]= rB+rB_n

In [15]:
seasonTeams.sort_values(by='rating',ascending=False,inplace=True)
seasonTeams.head(4)

,Team_Id,Team_Name,rating,games,wins,rating_0,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9,rating_10,rating_11
Team_Id,,,,,,,,,,,,,,,,,
1385,1385,St John's,96.603366,30,27,0.0,336.289074,361.562345,332.210810,280.764779,208.757078,144.993507,117.400102,105.461524,100.281072,98.024312,97.037064
1207,1207,Georgetown,96.134753,27,25,0.0,387.969596,406.539420,365.311357,301.745180,219.424515,149.775928,119.331055,106.059412,100.268062,97.734898,96.623692
1276,1276,Michigan,91.683013,28,25,0.0,361.720639,381.952925,345.012930,286.217942,208.853603,142.902369,114.005282,101.326022,95.741885,93.271471,92.173401
1424,1424,UNLV,90.924548,30,27,0.0,292.108629,307.709035,281.367511,239.420008,180.701812,127.859070,105.998618,97.011141,93.321058,91.805155,91.181554


In [16]:
seasonTeams.reset_index(drop=True).reset_index().head(4)

,index,Team_Id,Team_Name,rating,games,wins,rating_0,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9,rating_10,rating_11
0,0,1385,St John's,96.603366,30,27,0.0,336.289074,361.562345,332.210810,280.764779,208.757078,144.993507,117.400102,105.461524,100.281072,98.024312,97.037064
1,1,1207,Georgetown,96.134753,27,25,0.0,387.969596,406.539420,365.311357,301.745180,219.424515,149.775928,119.331055,106.059412,100.268062,97.734898,96.623692
2,2,1276,Michigan,91.683013,28,25,0.0,361.720639,381.952925,345.012930,286.217942,208.853603,142.902369,114.005282,101.326022,95.741885,93.271471,92.173401
3,3,1424,UNLV,90.924548,30,27,0.0,292.108629,307.709035,281.367511,239.420008,180.701812,127.859070,105.998618,97.011141,93.321058,91.805155,91.181554


In [17]:
seasonSeeds= tSeed[tSeed['Season']==1985]
seasonSeeds.head(4)

,Season,Seed,Team
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260


In [18]:
seasonTeamsOut= seasonSeeds.merge(seasonTeams, how='outer', indicator=True, left_on=['Team'], right_on=['Team_Id'])[list(seasonTeams.columns)+['Seed']]
seasonTeamsOut= seasonTeamsOut.rename(columns={'Team_Id':'Team'})
seasonTeamsOut.head(4)

,Team,Team_Name,rating,games,wins,rating_0,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9,rating_10,rating_11,Seed
0,1207,Georgetown,96.134753,27,25,0.0,387.969596,406.539420,365.311357,301.745180,219.424515,149.775928,119.331055,106.059412,100.268062,97.734898,96.623692,W01
1,1210,Georgia Tech,70.946808,31,24,0.0,278.748660,295.068189,266.273933,220.778353,161.207911,110.383800,88.092943,78.324172,74.035752,72.148385,71.315436,W02
2,1228,Illinois,61.832758,31,23,0.0,228.646965,249.812797,229.216013,191.980801,141.202550,97.011645,77.386669,68.648190,64.740450,62.984716,62.192155,W03
3,1260,Loyola-Chicago,78.541146,30,25,0.0,289.634569,298.523266,268.864011,224.551918,165.951168,115.357757,94.021901,85.010474,81.184852,79.550044,78.846352,W04


In [19]:
seasonTeamsOut.to_csv('../ELO_seasonTeams.csv')
seasonDataTeam.to_csv('../ELO_seasonDataTeam.csv')

In [20]:
lkajsdflkjs

NameError: name 'lkajsdflkjs' is not defined

In [ ]:
plt.plot(seasonTeams.sort_values(by='rating').reset_index(drop=True).index,seasonTeams.sort_values(by='rating')['rating'],'-')
plt.plot(seasonTeams.sort_values(by='rating_0').reset_index(drop=True).index,seasonTeams.sort_values(by='rating_0')['rating_0'],'-')
#plt.plot(seasonTeams.sort_values(by='rating_1').reset_index(drop=True).index,seasonTeams.sort_values(by='rating_1')['rating_1'],'-')
#plt.plot(seasonTeams.sort_values(by='rating_2').reset_index(drop=True).index,seasonTeams.sort_values(by='rating_2')['rating_2'],'-')
#plt.plot(seasonTeams.sort_values(by='rating_3').reset_index(drop=True).index,seasonTeams.sort_values(by='rating_3')['rating_3'],'-')
#plt.plot(seasonTeams.sort_values(by='rating_4').reset_index(drop=True).index,seasonTeams.sort_values(by='rating_4')['rating_4'],'-')

In [ ]:
plt.plot(seasonTeams['Team_Id'],seasonTeams['rating'],'.')
plt.plot(seasonTeams['Team_Id'],seasonTeams['rating_0'],'.')
#plt.plot(seasonTeams['Team_Id'],seasonTeams['rating_1'],'.')
#plt.plot(seasonTeams['Team_Id'],seasonTeams['rating_2'],'.')
#plt.plot(seasonTeams['Team_Id'],seasonTeams['rating_3'],'.')
#plt.plot(seasonTeams['Team_Id'],seasonTeams['rating_4'],'.')

In [ ]:
w= seasonTeams.sort_values(by='rating')['Team_Id'][:10]
l= seasonTeams.sort_values(by='rating', ascending=False)['Team_Id'][:10]
for i,t in enumerate(w.append(l)):
    plotData= seasonDataTeam[seasonDataTeam['team']==t]
    plt.plot(plotData['Daynum'],plotData['rating_pre'])
    #if (i+1) % 5 == 0: plt.show()
    #if i>= 10: break

In [ ]:
seasonTeams.loc[1228]

In [ ]:
seasonTeams.head(2)

In [ ]:
for i,t in enumerate(seasonSeeds['Team']):
    #plotData= x[x['Team']==t].sort_values(by='Daynum')
    plotData= seasonDataTeam[seasonDataTeam['team']==t].sort_values(by='Daynum')
    plt.plot(plotData['Daynum'],plotData['rating_pre'],label=t)
    if i >= 10: break

In [ ]:
#seasonSeeds.merge(seasonTeams, left_on='Team', right_on='Team_Id')
x= seasonSeeds.merge(seasonDataTeam, how='outer', left_on=['Team','Season'], right_on=['team','Season'], indicator=True)
print(x['_merge'].unique())
for i,t in enumerate(x[x['_merge']=='right_only']['team'].unique()):
    plotData= x[x['team']==t]
    plt.plot(plotData['Daynum'],plotData['rating_pre'])
    if i >= 10: break

In [ ]:
x.head(4)

In [ ]:
q= seasonDataTeam_orig
r= seasonDataTeam_orig

In [ ]:
cols= [u'teamGameID', u'gameID', u'Season', u'Daynum', u'team', u'outcome']
q= q[cols]
r= r[cols]

In [ ]:
#s= pd.concat([q,r], keys=[0,1], names=['round','gameID','outcome'])
s= pd.concat([q for i in range(2)], keys=[0,1], names=['round','gameID','outcome'])
s['rating_pre']= np.NAN
s['rating_post']= np.NAN
s.head(8)

In [ ]:
s.index.levels

In [ ]:
rnd=1
game=1
outcome=1
s.loc[[(rnd,game,outcome)],['rating_pre']]= 1400.
s.loc[[(rnd,game,outcome)],['rating_post']]= 1450.
s.loc[[(rnd,game,outcome)]]

In [ ]:
pd.MultiIndex.from_product([[0,1],[0,1,2],[0,1]])

In [ ]:
s.xs(0,level='round').head(4)

In [ ]:
s.xs(0,level='gameID').head(4)

In [ ]:
s.xs(0,level='outcome').head(4)

In [ ]:
seasonData.head(4)

In [ ]:
seasonData.set_index(['Season','gameID'], drop=True).head(4)

In [ ]:
seasonData= rscr[rscr['Season']==selectedSeason]
seasonData.sort_values(by=['Daynum','Wteam'])
seasonData.reset_index(inplace=True)
seasonData= seasonData.rename(columns={'index':'gameID'})
seasonData.head(4)

In [ ]:
r= rscr.reset_index().set_index(['Season','index'])
r.index.names= ['Season','gameID']
r.head(4)

In [ ]:
r.xs(2000,level='Season', drop_level=False).head(4)

In [ ]:
seasonTeams.head(4)

In [ ]:
def getTeamFrame(i):
    qq= seasonTeams_orig.set_index(np.repeat(i,len(seasonTeams_orig)), append=True)
    qq['rating']= np.NAN
    qq.index.names= ['Team_Id','round']
    qq.head(4)
    return qq

In [ ]:
#pd.concat([seasonTeams_orig,getTeamFrame(0)], axis=1)
pd.concat([getTeamFrame(0),getTeamFrame(1)], axis=0).sort_values(by='Team_Id').head(4)

In [ ]:
q= getTeamFrame(0)
q['rating']= 0
q.head()

In [ ]:
q.loc[1102]

In [ ]:
seasonData.head(4)

In [ ]:
for i,game in enumerate(range(len(seasonData))):
    x= seasonData.iloc[game]
    